In [1]:
import pandas as pd

In [14]:
data=pd.read_csv('Twitter_Data/Twitter_Data.csv')

In [15]:
data=data.dropna(subset=['category'])

In [17]:
data=data.drop(data[data['category']==0].index)

In [18]:
data.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
8,with upcoming election india saga going import...,1.0


In [19]:
data['category']=data['category'].replace(-1,0)

In [20]:
data.category.unique()

array([0., 1.])

In [25]:
data[data['category']==1].count()

clean_text    72249
category      72250
dtype: int64

In [26]:
data.to_csv('Twitter_Data/cleaned.csv')

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107760 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  107758 non-null  object 
 1   category    107760 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [28]:
# Filtering out the warnings
import warnings
warnings.filterwarnings("ignore")
#Load the necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk as nltk

In [29]:
data=data.rename(columns={'category': 'label'})

,clean_text,label
0,when modi promised “minimum government maximum...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
8,with upcoming election india saga going import...,1.0
...,...,...
162972,engine growth modi unveils indias first 12000 ...,1.0
162973,modi promised 2014 lok sabha elections that be...,1.0
162975,why these 456 crores paid neerav modi not reco...,0.0
162976,dear rss terrorist payal gawar what about modi...,0.0


In [31]:
data=data.rename(columns={'clean_text': 'text'})
data.columns

Index(['text', 'label'], dtype='object')

In [32]:
import re
import pandas as pd

# Function to remove non-English characters, handling NaN and other data types
def remove_non_english(text):
    """
    Removes non-English characters from a text string, handling NaN and other data types.

    Args:
        text: The input text string.

    Returns:
        The text string with non-English characters removed.
    """
    # Convert to string and replace NaN with an empty string
    text = str(text)
    text = text.replace('nan', '')  # Replace NaN values with an empty string
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Assuming you have your data in a pandas DataFrame called 'data' with a column 'Text'
# Apply the function to the 'Text' column
data['text'] = data['text'].apply(remove_non_english)

In [38]:
import string
string.punctuation
stopwords_En = nltk.corpus.stopwords.words('english')
#Expectation no and not
stopwords_En.remove('no')
stopwords_En.remove('not')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize #import nltk

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
wn = nltk.WordNetLemmatizer()

def clean_text(text):
    # Check if text is a string before processing
    if isinstance(text, str):
        text = "".join([word.lower() for word in text if word not in string.punctuation])
        tokens = word_tokenize(text)
        text = " ".join([wn.lemmatize(word) for word in tokens if word not in stopwords_En])
        return text
    # If text is not a string (e.g., float), return it as is or handle it appropriately
    else:
        return str(text)  # or return "" or handle NaN values differently

data['cleaned_Text'] = data['text'].apply(lambda x: clean_text(x))

In [41]:
data.to_csv('Twitter_Data/preprocessed.csv')

In [42]:
data.shape

(107760, 3)

In [45]:
import mlflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment("Sentiment_analysis_DEPI")

<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1729029343950, experiment_id='0', last_update_time=1729033419636, lifecycle_stage='active', name='Sentiment_analysis_DEPI', tags={}>

In [56]:
logged_model = 'runs:/fbd8d80ce1514764b6301ee1058a5e74/neural_network_classifier_model'

# Load model as a PyFuncModel.
model = mlflow.pyfunc.load_model(logged_model)


In [57]:
from sentence_transformers import SentenceTransformer

In [58]:
model_bert = SentenceTransformer('all-MiniLM-L6-v2')

In [106]:
from keras.models import load_model
import os
def load_sentiment_model():
    model_path = '../Amazon_review_sent_analysis model.h5'
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at {model_path}")
    model = load_model(model_path)
    return model
mymodel=load_sentiment_model()

In [141]:
def predict_sentiment(text):
    # Encode the input text using the BERT model
    embedding = model_bert.encode([text])

    # Make prediction using the loaded model
    # prob = model.predict(embedding)[0][0]  # Get the probability score
    prob2 = mymodel.predict(embedding)[0][0]  # Get the probability score


    # Determine the label based on the probability
    # label1 = "Positive" if prob > 0.5 else "Negative"
    label2 = "Positive" if prob2 > 0.5 else "Negative"

    # Return both the label and probability
    return label2,  prob2

In [144]:
# sample=data['cleaned_Text'][0]
sample='love'
xx=predict_sentiment(sample)
print(xx)
print(sample)

1/1 [==============================] - 0s 23ms/step
('Negative', 0.007748674)
love


In [50]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import mlflow
import mlflow.tensorflow
import mlflow
import mlflow.sklearn

In [108]:
data.head()

,text,label,cleaned_Text
0,when modi promised minimum government maximum ...,0.0,modi promised minimum government maximum gover...
2,what did just say vote for modi welcome bjp t...,1.0,say vote modi welcome bjp told rahul main camp...
3,asking his supporters prefix chowkidar their n...,1.0,asking supporter prefix chowkidar name modi gr...
4,answer who among these the most powerful world...,1.0,answer among powerful world leader today trump...
8,with upcoming election india saga going import...,1.0,upcoming election india saga going important p...


In [112]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_words = 5000
max_len=50

def tokenize_pad_sequences(text):
    '''
    This function tokenize the input text into sequnences of intergers and then
    pad each sequence to the same length
    '''
    # Text tokenization
    tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
    tokenizer.fit_on_texts(text)
    # Transforms text to a sequence of integers
    X = tokenizer.texts_to_sequences(text)
    # Pad sequences to the same length
    X = pad_sequences(X, padding='post', maxlen=max_len)
    # return sequences
    return X, tokenizer

print('Before Tokenization & Padding \n', data['cleaned_Text'][0])
X, tokenizer = tokenize_pad_sequences(data['cleaned_Text'])
print('\nAfter Tokenization & Padding \n', X[0])

Before Tokenization & Padding 
 modi promised minimum government maximum goverce expected begin difficult job reforming state take year get justice state not business exit psus temple

After Tokenization & Padding 
 [   1  278  668   30 1804  681  799 1676  871   60  102   43   15   26
  926  102    3  397 3664 4747 1142    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [113]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [116]:
from sklearn.model_selection import train_test_split
y = pd.get_dummies(data['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
print('Train Set ->', X_train.shape, y_train.shape)
print('Validation Set ->', X_val.shape, y_val.shape)
print('Test Set ->', X_test.shape, y_test.shape)

Train Set -> (64656, 50) (64656, 2)
Validation Set -> (21552, 50) (21552, 2)
Test Set -> (21552, 50) (21552, 2)


In [117]:
import keras.backend as K

def f1_score(precision, recall):
    ''' Function to calculate f1 score '''
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [125]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.metrics import Precision, Recall
from keras.optimizers import SGD
from keras import datasets

from keras.callbacks import LearningRateScheduler
from keras.callbacks import History
from tensorflow.keras.optimizers.legacy import SGD

from keras import losses

vocab_size = 5000
embedding_size = 32
epochs = 20
learning_rate = 0.01  # Reduced learning rate for stability
decay_rate = learning_rate / epochs
momentum = 0.9

# Create SGD optimizer with reduced learning rate and higher momentum
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
# Build model
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(32, return_sequences=True)))  # Add return_sequences for stacking
model.add(Bidirectional(LSTM(32)))  # Add another LSTM layer for deeper learning
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=sgd, 
               metrics=['accuracy', Precision(), Recall()])

# Train model

batch_size = 64
history = model.fit(X_train, y_train,
                      validation_data=(X_val, y_val),
                      batch_size=batch_size, epochs=epochs, verbose=1)

loss, accuracy, precision, recall = model.evaluate(X_test, y_test, verbose=0)
# Print metrics
print('')
print('Accuracy  : {:.4f}'.format(accuracy))
print('Precision : {:.4f}'.format(precision))
print('Recall    : {:.4f}'.format(recall))
print('F1 Score  : {:.4f}'.format(f1_score(precision, recall)))

model.save('lstm.h5')


In [126]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer=sgd, 
               metrics=['accuracy', Precision(), Recall()])

# Train model

batch_size = 64
history = model.fit(X_train, y_train,
                      validation_data=(X_val, y_val),
                      batch_size=batch_size, epochs=epochs, verbose=1)



Epoch 1/20
1011/1011 [==============================] - 33s 25ms/step - loss: 0.6355 - accuracy: 0.6694 - precision_1: 0.6694 - recall_1: 0.6694 - val_loss: 0.6320 - val_accuracy: 0.6723 - val_precision_1: 0.6723 - val_recall_1: 0.6723
Epoch 2/20
1011/1011 [==============================] - 23s 23ms/step - loss: 0.6337 - accuracy: 0.6700 - precision_1: 0.6700 - recall_1: 0.6700 - val_loss: 0.6313 - val_accuracy: 0.6723 - val_precision_1: 0.6723 - val_recall_1: 0.6723
Epoch 3/20
1011/1011 [==============================] - 23s 22ms/step - loss: 0.6327 - accuracy: 0.6700 - precision_1: 0.6700 - recall_1: 0.6700 - val_loss: 0.6309 - val_accuracy: 0.6723 - val_precision_1: 0.6723 - val_recall_1: 0.6723
Epoch 4/20
1011/1011 [==============================] - 23s 23ms/step - loss: 0.6318 - accuracy: 0.6700 - precision_1: 0.6700 - recall_1: 0.6700 - val_loss: 0.6296 - val_accuracy: 0.6723 - val_precision_1: 0.6723 - val_recall_1: 0.6723
Epoch 5/20
1011/1011 [==============================] - 

In [127]:
loss, accuracy, precision, recall = model.evaluate(X_test, y_test, verbose=0)
# Print metrics
print('')
print('Accuracy  : {:.4f}'.format(accuracy))
print('Precision : {:.4f}'.format(precision))
print('Recall    : {:.4f}'.format(recall))
print('F1 Score  : {:.4f}'.format(f1_score(precision, recall)))


Accuracy  : 0.8995
Precision : 0.8995
Recall    : 0.8995
F1 Score  : 0.8995


In [128]:
model.save('lstm.h5')

In [172]:
from keras.models import load_model
import mlflow

logged_model = 'runs:/f8b0e6e1aee14f4f89c1b68e3e56bfc8/lstm'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)



In [185]:
def predict_class(text):
    '''Function to predict sentiment class of the passed text'''
    
    sentiment_classes = ['Negative', 'Positive']
    max_len=50
    
    # Transforms text to a sequence of integers using a tokenizer object
    xt = tokenizer.texts_to_sequences(text)
    # Pad sequences to the same length
    xt = pad_sequences(xt, padding='post', maxlen=max_len)
    # Do the prediction using the loaded model
    yt = loaded_model.predict(xt).argmax(axis=1)
    # Print the predicted sentiment
    print('The predicted sentiment is', sentiment_classes[yt[0]])

In [179]:
def predict_class(text):
    '''Function to predict sentiment class of the passed text and return probability'''
    
    sentiment_classes = ['Negative', 'Positive']
    max_len = 50
    
    # Transforms text to a sequence of integers using a tokenizer object
    xt = tokenizer.texts_to_sequences(text)
    # Pad sequences to the same length
    xt = pad_sequences(xt, padding='post', maxlen=max_len)
    
    # Do the prediction using the loaded model
    probabilities = loaded_model.predict(xt)
    
    # Get the predicted class (with highest probability)
    predicted_class = probabilities.argmax(axis=1)[0]
    
    # Print the predicted sentiment and the corresponding probability
    predicted_sentiment = sentiment_classes[predicted_class]
    predicted_probability = probabilities[0][predicted_class]
    
    print(f"The predicted sentiment is '{predicted_sentiment}' with probability {predicted_probability:.4f}")
    
    return predicted_sentiment, predicted_probability


In [187]:
predict_class(['I love india'])

1/1 [==============================] - 0s 27ms/step
The predicted sentiment is Positive
1/1 [==============================] - 0s 29ms/step


0.005082759